## Table of Contents
* [Aerotech Data Collection Platform](#Aerotech-Data-Collection-Platform)
    * [Variables](#Variables:)
    * [Functions](#Functions:)
    * [Convert .gprec File](#Convert-.gprec-3D-scans-to-.csv-for-better-readability-(Do-not-run-if-files-already-converted):)
    * [Select Experiment Data to View](#Select-Experiment-Data-to-View:)
    * [Load Scan File](#Load-Scan-File:)
    * [Display 3D Scan](#Display-3D-Scan:)
    * [Import Feedback Data](#Import-Feedback-Data:)
    * [Extract Signals From Feedback](#Extract-Signals-from-Feedback-Data:)
    * [Fill Missing Pixel From 3D Scan Data](#Fill-missing-pixel-data-from-3D-scan:)
    * [Create Mask Surrounding Bead](#Create-mask-of-the-area-surrounding-the-bead:)
    * [Remove Distortion From Scan](#Remove-twist-from-3D-scan-substrate-for-accurate-bead-area-calculation:)
    * [Calculate Bead Cross Secional Area](#Calculate-Bead-Cross-Sectional-Area:)
    * [Resample Bead Area](#Resample-Bead-Area-To-Fit-Feedback-Data:)
#### [🏠 Home](../../../welcomePage.ipynb)

# Aerotech Data Collection Platform 
This script describes preprocessing steps and techniques for the preparation of sensor feedback data for data-driven modeling techniques. The data is generated on a custom-built Fused Filament Fabrication (FFF) 3D printer. The highly instrumented 3D printer was built to collect a large quantity of data for the purpose of data-driven modeling of the extrusion process. The 3D printer is based on Aerotech motion components with the capability of collecting feedback signals that include position, velocity, acceleration, and motor current. Additional sensors were placed where the dynamics of the system are expected to be visible. The sensors include a polymer melt pressure sensor, a thermocouple for melt temperature, a load cell to measure force on the filament feedstock, and a thermal camera to monitor cooling dynamics post-extrusion. A figure of the system is shown below:


<img src="instrumentedSystemBuild_UpdatedSystem.png" alt="Aerotech System" width="400" height="300"/> <img src="extruderDetails.png" alt="Extruder Close up" width="400" height="300"/>


## Variables:
This section involves the selection of the folder where the experiment feedback data is saved by the Aerotech system, as well as all the other variables used in this script. After each experiment, all the experiment data is placed in a single folder. The folder contains the following files:

1. comma-separated values (.csv) file with Aerotech sensor feedback signals.

2. Gocator recording (.gprec) file with laser profilometer 3D height data.

3. raw audio video interleave (.ravi) file with thermal camera recording of the experiment.

#### Press ▶️ <font color = '#2195F2'>**Play**</font> Button to Import Libraries and Load Variables

In [ ]:
print("🟧 Running", end='\r')
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import re
import csv
from scipy.interpolate import interp1d
import ipywidgets as widgets
from IPython.display import display, clear_output

mainExperimentsFolder = r"/home/jovyan/training-course/Modules/3. Advanced Topics/3.1. Aerotech Data Collection Platform/data"
#ConverterToolFilePath = r"C:\Users\Sam\OneDrive - Umich\PannierResearchLab\Osama Habbal's Research\physicsInformedDataDrivenModeling\Code\ReplayConverter\ReplayConverter.exe"
maskThreshold = -0.225
#experimentScanNumber = 0

print("✅ Success ")

## Functions:
This section contains all the functions that are used in this notebook, we will go over the details of these functions in their respective section.

#### Press ▶️ <font color = '#2195F2'>**Play**</font> Button to Load Script Functions

In [24]:
print("🟧 Running", end='\r')
# Function to extract the number from a file name
def extractNumber(file_name):
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else 0

def listFiles(fileDirectory):
    files = [file for file in os.listdir(fileDirectory) ]
    numExperiments =  len(files)
    files = sorted(files, key=extractNumber)
    return files, numExperiments

def convertScanToCsv(gprecFilePath,csvFileName,ConverterToolFilePath):
    # Arguments to be passed to the program
    arguments = "ReplayConverter.exe -i " + gprecFilePath + " -a -o " + csvFileName
    #arguments = "ReplayConverter.exe -i " + gprecFilePath + " -o " + csvFileName
    command = f'{arguments}'
    #change directory to where the tool is
    os.chdir(ConverterToolFilePath[:-20])
    # Run the program
    res = os.system(command)
    return command,res
def loadScanCsv(scanFilePath,filename):
    #Go to Scan Directory
    os.chdir(scanFilePath)
    # Initialize an empty list to store rows
    scannedGeometery = []
    
    # Open the CSV file and manually read it line by line
    with open(filename, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        
        # Process each row
        for row in csvreader:
            # Append the row to the data list
            scannedGeometery.append(row)
    
    # Find the maximum row length
    max_len = max(len(row) for row in scannedGeometery)
    
    # Pad shorter rows with NaNs to make all rows the same length
    for row in scannedGeometery:
        while len(row) < max_len:
            row.append(None)
    
    # Create a DataFrame from the data
    scannedGeometeryDataFrame = pd.DataFrame(scannedGeometery)
    #Extract Data from scan csv file:
    xAxisArrayTemp = scannedGeometeryDataFrame.iloc[27,1:].to_numpy()
    yAxisArrayTemp = scannedGeometeryDataFrame.iloc[28:2275,0].to_numpy()
    heightMapTemp = scannedGeometeryDataFrame.iloc[28:,1:].to_numpy()
    scanWidth = np.shape(heightMapTemp)[1]
    scanLength = np.shape(heightMapTemp)[0]
    heightMap = heightMapTemp[int((scanLength*0.5)-1000):int((scanLength*0.5)+1000),int((scanWidth*0.5)-300):int((scanWidth*0.5)+120)]
    xAxisArray = xAxisArrayTemp[int((scanWidth*0.5)-300):int((scanWidth*0.5)+120)]
    yAxisArray = yAxisArrayTemp[int((scanLength*0.5)-1000):int((scanLength*0.5)+1000)]
    heightMap[heightMap == ''] = np.nan
    heightMap[heightMap == None] = np.nan
    return xAxisArray, yAxisArray, heightMap
    
# Function to check and delete if the first cell in a row starts with a semicolon
def remove_first_semicolon(df):
    for index, row in df.iterrows():
        #print(row.iloc[0])
        #print(row)
        if row.iloc[0].startswith('; '):
            df.iloc[index, 0] = ''  # Replace with empty string or None if deletion is necessary
            # Shift remaining cells to the left
            df.iloc[index] = df.iloc[index].shift(-1).fillna('')
            print(row.iloc[0])
    return df
    
# def loadFeedbackDataCsv(dataFilePath,filename):
#     #Go to Scan Directory
#     os.chdir(dataFilePath)
#     data = pd.read_csv(filename,skiprows=3,sep = ' ', engine='python')
#     data = data.replace('; ','',regex=True)
#     #data.columns = data.columns[1:]
#     #data = remove_first_semicolon(data)
#     return data


def loadFeedbackDataCsv(dataFilePath,filename):
    #Go to Scan Directory
    os.chdir(dataFilePath)    
    # Read the file, skip the initial '; ' in the header
    with open(filename, 'r') as file:
        lines = file.readlines()[3:]
        print(np.shape(lines))
        # Find the line containing '[File]' and get the index
        file_line_index = next((index for index, line in enumerate(lines) if '; [File]' in line), len(lines))
        # Slice the list to keep only the lines before ';[File]'
        lines = lines[:file_line_index-1]
        header = lines[0].lstrip('; ').strip().split()  # Remove the '; ' and split by spaces
        data = [line.strip().split() for line in lines[1:]]
    
    # Create the DataFrame
    df = pd.DataFrame(data, columns=header)
    return df

def extractFeedbackSignals(experimentFeedbackData):
    experimentTimeArray = np.array(experimentFeedbackData['SampleTime[0]'], dtype=float)
    experimentTimeArray = experimentTimeArray-experimentTimeArray[0] 
    xAxisPositionCommand = np.array(experimentFeedbackData['PosCmd#00[0]'], dtype=float)/50000
    xAxisPositionFeedback = np.array(experimentFeedbackData['PosFbk#00[0]'], dtype=float)/50000
    xAxisVelocityCommand = np.array(experimentFeedbackData['VelCmd#00[0]'], dtype=float)
    xAxisVelocityFeedback = np.array(experimentFeedbackData['VelFbk#00[0]'], dtype=float)
    xAxisAccelerationCommand = np.array(experimentFeedbackData['AccCmd#00[0]'], dtype=float)
    xAxisAccelerationFeedback = np.array(experimentFeedbackData['AccFbk#00[0]'], dtype=float)
    loadCellVoltageSignal = np.array(experimentFeedbackData['Ain_0#00[0]'], dtype=float)
    thermocoupleVoltageSignal = np.array(experimentFeedbackData['Ain_1#00[0]'], dtype=float)
    pressureSensorVoltageSignal = np.array(experimentFeedbackData['Ain_2#00[0]'], dtype=float)
    yAxisPositionCommand = np.array(experimentFeedbackData['PosCmd#01[0]'], dtype=float)
    yAxisPositionFeedback = np.array(experimentFeedbackData['PosFbk#01[0]'], dtype=float)
    yAxisVelocityCommand = np.array(experimentFeedbackData['VelCmd#01[0]'], dtype=float)
    yAxisVelocityFeedback = np.array(experimentFeedbackData['VelFbk#01[0]'], dtype=float)
    yAxisAccelerationCommand = np.array(experimentFeedbackData['AccCmd#01[0]'], dtype=float)
    yAxisAccelerationFeedback = np.array(experimentFeedbackData['AccFbk#01[0]'], dtype=float)
    extruderPositionCommand = np.array(experimentFeedbackData['PosCmd#04[0]'], dtype=float)
    extruderPositionFeedback = np.array(experimentFeedbackData['PosFbk#04[0]'], dtype=float)
    extruderVelocityCommand = np.array(experimentFeedbackData['VelCmd#04[0]'], dtype=float)
    extruderVelocityFeedback = np.array(experimentFeedbackData['VelFbk#04[0]'], dtype=float)
    extruderAccelerationCommand = np.array(experimentFeedbackData['AccCmd#04[0]'], dtype=float)
    extruderAccelerationFeedback = np.array(experimentFeedbackData['AccFbk#04[0]'], dtype=float)
    extruderMotorCurrentCommand = np.array(experimentFeedbackData['CurCmd#04[0]'], dtype=float)
    extruderMotorCurrentFeedback = np.array(experimentFeedbackData['CurFbk#04[0]'], dtype=float)
    extruderMotorAverageCurrentFeedback = np.array(experimentFeedbackData['CurFbkAvg#04[0]'], dtype=float)
    filamentPositionFeedback = np.array(experimentFeedbackData['PosFbkAux#04[0]'], dtype=float)
    filamentPositionFeedback = filamentPositionFeedback - filamentPositionFeedback[0]
    if 'PosCmd#03[0]' in experimentFeedbackData.columns:
        zAxisPositionCommand = np.array(experimentFeedbackData[r'PosCmd#03[0]'], dtype=float)
        zAxisPositionFeedback = np.array(experimentFeedbackData[r'PosFbk#03[0]'], dtype=float)
        zAxisVelocityCommand = np.array(experimentFeedbackData[r'VelCmd#03[0]'], dtype=float)
        zAxisVelocityFeedback = np.array(experimentFeedbackData[r'VelFbk#03[0]'], dtype=float)
        zAxisAccelerationCommand = np.array(experimentFeedbackData[r'AccCmd#03[0]'], dtype=float)
        zAxisAccelerationFeedback = np.array(experimentFeedbackData[r'AccFbk#03[0]'], dtype=float)
    else:
        zAxisPositionCommand = None
        zAxisPositionFeedback = None
        zAxisVelocityCommand = None
        zAxisVelocityFeedback = None
        zAxisAccelerationCommand = None
        zAxisAccelerationFeedback = None
        print(f"Warning: Z position, velocity, acceleration data were not found and will be ignored")

    return experimentTimeArray,xAxisPositionCommand,xAxisPositionFeedback,xAxisVelocityCommand,xAxisVelocityFeedback,xAxisAccelerationCommand,xAxisAccelerationFeedback,loadCellVoltageSignal,thermocoupleVoltageSignal,pressureSensorVoltageSignal,yAxisPositionCommand,yAxisPositionFeedback,yAxisVelocityCommand,yAxisVelocityFeedback,yAxisAccelerationCommand, yAxisAccelerationFeedback,extruderPositionCommand,extruderPositionFeedback,extruderVelocityCommand,extruderVelocityFeedback,extruderAccelerationCommand,extruderAccelerationFeedback,extruderMotorCurrentCommand,extruderMotorCurrentFeedback,extruderMotorAverageCurrentFeedback,filamentPositionFeedback,zAxisPositionCommand,zAxisPositionFeedback,zAxisVelocityCommand,zAxisVelocityFeedback,zAxisAccelerationCommand,zAxisAccelerationFeedback

def fillScan(height_map):
    # Ensure the height_map is a numpy array of type float
    height_map = np.array(height_map, dtype=float)
    
    # Create a copy of the height_map to be filled
    filled_height_map = np.copy(height_map)
    
    m, n = height_map.shape
    
    # Loop through each element of the height map
    for i in range(m):
        for j in range(n):
            # Check if the current element is NaN
            if np.isnan(filled_height_map[i, j]):
                # Find the surrounding 8 pixels
                neighbors = []
                
                if i > 0:
                    neighbors.append(filled_height_map[i-1, j])
                    if j > 0:
                        neighbors.append(filled_height_map[i-1, j-1])
                    if j < n - 1:
                        neighbors.append(filled_height_map[i-1, j+1])
                
                if i < m - 1:
                    neighbors.append(filled_height_map[i+1, j])
                    if j > 0:
                        neighbors.append(filled_height_map[i+1, j-1])
                    if j < n - 1:
                        neighbors.append(filled_height_map[i+1, j+1])
                
                if j > 0:
                    neighbors.append(filled_height_map[i, j-1])
                
                if j < n - 1:
                    neighbors.append(filled_height_map[i, j+1])
                
                # Remove NaN values from neighbors before calculating the average
                neighbors = [value for value in neighbors if not np.isnan(value)]
                
                # Calculate the average value of the surrounding pixels
                if neighbors:
                    avg_value = np.nanmean(neighbors)
                else:
                    avg_value = np.nan
                
                # Replace the current element with the average value
                filled_height_map[i, j] = avg_value
    
    return filled_height_map
def createMask(height_map, threshold):
    """
    Creates a mask for height values less than the specified threshold.
    
    Parameters:
    height_map (array-like): Input height map.
    threshold (float): Threshold value.
    
    Returns:
    np.ndarray: Mask with True where height values are less than the threshold, False otherwise.
    """
    z_mask = np.zeros_like(height_map)
    
    # Loop through each experiment
    # Apply the threshold condition
    mask = height_map[:, :] < threshold
    
    return mask

def levelScan(trimmed_scan_z_position, z_mask, trimmed_scan_x_position, filled_trimmed_scan_z_position):
    """
    Levels the scan positions by fitting a polynomial and subtracting it from the z-position.
    Parameters:
    trimmed_scan_z_position (np.ndarray): Array of trimmed scan z-positions.
    z_mask (np.ndarray): Array of z-mask values.
    trimmed_scan_x_position (np.ndarray): Array of trimmed scan x-positions.
    filled_trimmed_scan_z_position (np.ndarray): Array of filled trimmed scan z-positions.
    Returns:
    np.ndarray: Array of leveled trimmed scan z-positions.
    """
    # Initialize the leveled_trimmed_scan_z_position array
    leveled_trimmed_scan_z_position = np.copy(filled_trimmed_scan_z_position)

    # Ensure all inputs are numpy arrays of type float
    trimmed_scan_z_position = np.array(trimmed_scan_z_position, dtype=float)
    z_mask = np.array(z_mask, dtype=float)
    trimmed_scan_x_position = np.array(trimmed_scan_x_position, dtype=float)
    filled_trimmed_scan_z_position = np.array(filled_trimmed_scan_z_position, dtype=float)
    
    # Loop through each line
    for line in range(trimmed_scan_z_position.shape[0]):
        # Find indices to keep based on non-zero values in z_mask
        indices_to_keep = np.nonzero(z_mask[line, :])[0]
        
        if indices_to_keep.size > 0:  # Ensure there are valid indices to keep
            # Fit a polynomial to the selected x and z positions
            p_fit = np.polyfit(trimmed_scan_x_position[indices_to_keep],
                               filled_trimmed_scan_z_position[line, indices_to_keep], 1)
            
            # Evaluate the polynomial at all x positions
            z_to_subtract_array = np.polyval(p_fit, trimmed_scan_x_position)
            
            # Subtract the polynomial fit from the z positions to level them
            leveled_trimmed_scan_z_position[line, :] = (
                filled_trimmed_scan_z_position[line, :] - z_to_subtract_array
            )
        else:
            # If no indices are found, set leveled_trimmed_scan_z_position to NaN or some default value
            leveled_trimmed_scan_z_position[line, :] = np.nan
    
    return leveled_trimmed_scan_z_position

def calculateBeadArea(trimmed_scan_z_position, z_mask, trimmed_scan_x_position, leveled_trimmed_scan_z_position):
    """
    Calculates the bead area by integrating the leveled z-positions over the x-positions.

    Parameters:
    trimmed_scan_z_position (np.ndarray): 3D array of trimmed scan z-positions.
    z_mask (np.ndarray): 3D array of z-mask values.
    trimmed_scan_x_position (np.ndarray): 3D array of trimmed scan x-positions.
    leveled_trimmed_scan_z_position (np.ndarray): 3D array of leveled trimmed scan z-positions.

    Returns:
    np.ndarray: 3D array of bead areas.
    """
    # Ensure all inputs are numpy arrays of type float
    trimmed_scan_z_position = np.array(trimmed_scan_z_position, dtype=float)
    z_mask = np.array(z_mask, dtype=bool)
    trimmed_scan_x_position = np.array(trimmed_scan_x_position, dtype=float)
    leveled_trimmed_scan_z_position = np.array(leveled_trimmed_scan_z_position, dtype=float)
    
    # Initialize the bead_area array with the same shape as leveled_trimmed_scan_z_position
    bead_area = np.zeros(leveled_trimmed_scan_z_position.shape[0])

    # Loop through each line
    for line in range(trimmed_scan_z_position.shape[0]):
        # Find bead indices based on z_mask being 0
        bead_indices = np.where(z_mask[line, :] == False)[0]

        if bead_indices.size > 0:  # Ensure there are valid bead indices
            y = leveled_trimmed_scan_z_position[line, bead_indices]
            x = trimmed_scan_x_position[bead_indices]
            #print(f"Line {line}: y = {y}, x = {x}")

            # Remove NaN values from y and corresponding x
            valid_indices = ~np.isnan(y)
            y = y[valid_indices]
            x = x[valid_indices]
            #print(f"Line {line}: filtered y = {y}, filtered x = {x}")

            if y.size > 0 and x.size > 0:  # Ensure there are still valid points to integrate
                bead_area[line] = np.trapz(y, x)
        if np.any(np.isnan(bead_area[line])):
            bead_area[line] = 0
    
    return bead_area

def resampleBeadArea(beadArea,axisScan,axisFeedbackData):
    # Convert arrays to numeric types, replacing non-numeric values with NaN
    beadCrossSectionArea = np.array(beadArea, dtype=np.float64)
    axisScan = np.array(axisScan, dtype=np.float64)
    axisFeedbackData = np.array(axisFeedbackData, dtype=np.float64)

    
    # Remove any NaN values if necessary (optional, based on your specific use case)
    beadCrossSectionArea = beadArea[~np.isnan(beadArea)]
    #axisScan = yAxisArray[~np.isnan(axisScan)]
    axisFeedbackData = axisFeedbackData[~np.isnan(axisFeedbackData)]
    # Create an interpolator function
    interpolator = interp1d(axisScan, beadArea, kind='linear', bounds_error=False, fill_value="extrapolate")
    # Perform the interpolation
    interpolatedBeadArea = interpolator(axisFeedbackData)
    return interpolatedBeadArea
    
# Function to update the plot
def updatePlot(timeAxis,selectionData,stringName):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=timeAxis/1000, y=selectionData, mode='lines', name=stringName, showlegend=True))
    fig.update_layout(
        title=f'Data for {stringName}',
        yaxis_title=stringName,
        xaxis_title='Time (s)'
    )
    clear_output(wait=True)
    display(dropdown)  # Redisplay the dropdown
    fig.show()

# Event handler for the dropdown menu
def onDropdownChange(change):
    selected_option = change['new']

print("✅ Success ")

## Convert .gprec 3D scans to .csv for better readability (Do not run if files already converted):
This section handles the conversion of the 3D profilometer proprietary (.gprec) files to a readable format, such as a (.csv) file, for all the experiments. The function finds all the experiment folders, locates each (.gprec) file, and then calls an executable program (.exe) that takes the information within the (.gprec) file and converts it into a (.csv) file. The executable is a utility tool provided by the manufacturer of the line profilometer (LMI3D). The output (.csv) file contains a 3D height map of the printed line. A height map is a mapping of z height at some x-y coordinate.

In [ ]:
print("🟧 Running", end='\r')
fileNames,numExperimentFiles = listFiles(mainExperimentsFolder)
print("✅ Success ")
print(fileNames,numExperimentFiles)
# try: 
#     for file in range(numExperimentFiles):
#         folderPath = mainExperimentsFolder+'\\' + fileNames[file]
#         os.chdir(folderPath)
#         gprecFilePath = mainExperimentsFolder+ '\\' +  fileNames[file]+ '\\' + 'EXP_'+str(file)+'.gprec'
#         csvFileName = mainExperimentsFolder+ '\\' + fileNames[file]+ '\\' + fileNames[file]+'_SCAN.csv'
#         outputFileName = fileNames[file]+'_SCAN.csv'
#         command,res = convertScanToCsv(gprecFilePath,csvFileName,ConverterToolFilePath)
# except:
#     print("Error occured or csv files already exist")

## Select Experiment Data to View:

#### Press ▶︎ to Select Experiment and View Data:

In [ ]:
experimentScanNumberTemp = widgets.IntSlider(
                            value=0,
                            min=0,
                            max=numExperimentFiles,
                            step=1,
                            description='Test:',
                            disabled=False,
                            continuous_update=False,
                            orientation='horizontal',
                            readout=True,
                            readout_format='d'
                        )
display(experimentScanNumberTemp)

## Load Scan File:
Load the (.csv) 3D height map

#### Press ▶︎ to Load Scan File:

In [46]:
print("🟧 Running", end='\r')
os.listdir()
experimentScanNumber = experimentScanNumberTemp.value
fileNames,numExperimentFiles = listFiles(mainExperimentsFolder)
scanFileName =  fileNames[experimentScanNumber]+'_SCAN.csv'
scanFilePath = mainExperimentsFolder+ '/' + fileNames[experimentScanNumber]
print("✅ Success ")
#Load CSV Scan:
xAxisArray, yAxisArray, heightMap = loadScanCsv(scanFilePath,scanFileName)

## Display 3D Scan:
The imported 3D height map is displayed as an image

#### Press ▶︎ to display Loaded Scan as an Image:

In [ ]:
# Create a figure with a specific size
fig, ax = plt.subplots(figsize=(1, 8))  # Width, height in inches

cax = plt.imshow(heightMap.astype(float), 
                    interpolation ='nearest', origin ='lower',cmap='viridis', aspect = 'auto')  # 'viridis' colormap
#cbar = plt.colorbar()  # Add a color bar for reference
cbar = plt.colorbar(fraction=0.5, pad=0.04) 
# Set the colorbar label
cbar.set_label('Height Value (mm)')

# Add x and y axis labels
plt.xlabel('Y Axis')
plt.ylabel('X Axis')

# Hide the x and y ticks
plt.xticks([])
plt.yticks([])
plt.show()

## Import Feedback Data:
This section imports the (.csv) file provided by the Aerotech system, which contains all the feedback signals from the system. The (.csv) file contains space-separated feedback signals sampled at a frequency of 1 kHz.

#### Press ▶︎ to Import Experiment Feedback Data:

In [ ]:
print("🟧 Running", end='\r')
experimentDataNumber = experimentScanNumber
fileNames,numExperimentFiles = listFiles(mainExperimentsFolder)
dataFileName =  fileNames[experimentDataNumber]+'.csv'
dataFilePath = mainExperimentsFolder+ '/' + fileNames[experimentDataNumber]
print("✅ Success ")
experimentFeedbackData = loadFeedbackDataCsv(dataFilePath,dataFileName)

## Extract Signals from Feedback Data:
In this section, Indiviual signals are extracted for further use. To view each of these signals in time domain, select from the dropdown menu which signal you would like to observe:

#### Press ▶︎ to Parse Experiment Feedback Data:

In [ ]:
print("🟧 Running", end='\r')
(
    experimentTimeArray, xAxisPositionCommand, xAxisPositionFeedback, 
    xAxisVelocityCommand, xAxisVelocityFeedback, xAxisAccelerationCommand, 
    xAxisAccelerationFeedback, loadCellVoltageSignal, thermocoupleVoltageSignal, 
    pressureSensorVoltageSignal, yAxisPositionCommand, yAxisPositionFeedback, 
    yAxisVelocityCommand, yAxisVelocityFeedback, yAxisAccelerationCommand, 
    yAxisAccelerationFeedback, extruderPositionCommand, extruderPositionFeedback, 
    extruderVelocityCommand, extruderVelocityFeedback, extruderAccelerationCommand, 
    extruderAccelerationFeedback, extruderMotorCurrentCommand, 
    extruderMotorCurrentFeedback, extruderMotorAverageCurrentFeedback, 
    filamentPositionFeedback, zAxisPositionCommand, zAxisPositionFeedback, 
    zAxisVelocityCommand, zAxisVelocityFeedback, zAxisAccelerationCommand, 
    zAxisAccelerationFeedback
) = extractFeedbackSignals(experimentFeedbackData)

# Define the options for the dropdown menu
options = ['xAxisPositionCommand', 'xAxisPositionFeedback', 
    'xAxisVelocityCommand', 'xAxisVelocityFeedback', 'xAxisAccelerationCommand', 
    'xAxisAccelerationFeedback', 'loadCellVoltageSignal', 'thermocoupleVoltageSignal', 
    'pressureSensorVoltageSignal', 'yAxisPositionCommand', 'yAxisPositionFeedback', 
    'yAxisVelocityCommand', 'yAxisVelocityFeedback', 'yAxisAccelerationCommand', 
    'yAxisAccelerationFeedback', 'extruderPositionCommand', 'extruderPositionFeedback', 
    'extruderVelocityCommand', 'extruderVelocityFeedback', 'extruderAccelerationCommand', 
    'extruderAccelerationFeedback', 'extruderMotorCurrentCommand', 
    'extruderMotorCurrentFeedback', 'extruderMotorAverageCurrentFeedback', 
    'filamentPositionFeedback', 'zAxisPositionCommand', 'zAxisPositionFeedback', 
    'zAxisVelocityCommand', 'zAxisVelocityFeedback', 'zAxisAccelerationCommand', 
    'zAxisAccelerationFeedback']

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    value=options[0],  # Default value
    description='Select:',
    disabled=False,
)

# Function to handle the selection change
def on_change(change):
    selected_value = change['new']


# Attach the function to the dropdown widget
dropdown.observe(on_change, names='value')
print("✅ Success ")

# Display the dropdown widget
display(dropdown)

#### Press ▶︎ to Update Plot:

In [ ]:
print("🟧 Running", end='\r')
updatePlot(experimentTimeArray,locals()[dropdown.value],dropdown.value)
print("✅ Success ")

## Fill missing pixel data from 3D scan:
The 3D height map of the experiment contains missing data points, which is a common occurrence with the type of sensors used in this system. To calculate the bead area, the missing points must be filled. The fillScan function takes a height map represented as a numpy array of floating-point values and ensures it's of the correct type. It creates a copy of the input height map to work with, preserving the original data. Using nested loops, it iterates through each element of the height map. When encountering NaN (Not a Number) values, it identifies the surrounding 8 pixels, calculates their average excluding NaNs, and replaces the NaN with this computed average. Finally, the function returns a filled height map where all NaN values have been replaced with appropriate average values derived from neighboring pixels.

#### Press ▶︎ to Fill Missing Points in 3D scan:

In [59]:
print("🟧 Running", end='\r')
filledScan = fillScan(heightMap)
print("✅ Success ")

#### Press ▶︎ to Display filled 3D scan:

In [ ]:
# Create a figure with a specific size
fig, ax = plt.subplots(figsize=(1, 8))  # Width, height in inches

cax = plt.imshow(filledScan.astype(float), 
                    interpolation ='nearest', origin ='lower',cmap='viridis', aspect = 'auto')  # 'viridis' colormap
#cbar = plt.colorbar()  # Add a color bar for reference
cbar = plt.colorbar(fraction=0.5, pad=0.04) 
# Set the colorbar label
cbar.set_label('Height Value (mm)')

# Add x and y axis labels
plt.xlabel('Y Axis')
plt.ylabel('X Axis')

# Hide the x and y ticks
plt.xticks([])
plt.yticks([])
plt.show()

## Create mask of the area surrounding the bead:
As the scanned substrate is not exactly flat, to accurately calculate the bead area, the scan should be leveled so that all of the substrate is at the zero ground level. To achieve this leveling, the first step is to create a mask to remove the actual bead and identify the edges of the bead. The createMask function generates a boolean mask from an input height map, marking True where height values are below a specified threshold and False otherwise. It initializes a zero-filled array of the same shape as the input height map. By applying a simple threshold comparison using numpy operations, it efficiently creates and returns the mask that identifies areas where the height values are less than the specified threshold.

#### Press ▶︎ to Create Mask :

In [61]:
print("🟧 Running", end='\r')
mask = createMask(filledScan, maskThreshold)
print("✅ Success ")

#### Press ▶︎ to Display Mask :

In [ ]:
# Create a figure with a specific size
fig, ax = plt.subplots(figsize=(1, 8))  # Width, height in inches

cax = plt.imshow(mask.astype(float), 
                    interpolation ='nearest', origin ='lower',cmap='viridis', aspect = 'auto')  # 'viridis' colormap
#cbar = plt.colorbar()  # Add a color bar for reference
cbar = plt.colorbar(fraction=0.5, pad=0.04) 
# Set the colorbar label
cbar.set_label('Height Value (mm)')

# Add x and y axis labels
plt.xlabel('Y Axis')
plt.ylabel('X Axis')

# Hide the x and y ticks
plt.xticks([])
plt.yticks([])
plt.show()

## Remove twist from 3D scan substrate for accurate bead area calculation:
The levelScan function aims to adjust or level scan positions by fitting a polynomial curve and subtracting it from the z-positions of a height map. It starts by initializing an array to store the leveled z-positions based on the height map data. All input arrays are converted to numpy arrays of type float to ensure consistency. For each line in the height map z-position array, the function identifies valid indices based on non-zero values in the z_mask, which indicates areas of interest. If valid indices are found, it fits a linear polynomial to these positions, evaluates this polynomial across all x positions, and subtracts the resulting curve from the filled height map z-positions to level them. If no valid indices are found, it assigns NaN or a default value to the leveled z-positions for that line. Finally, the function returns the array of leveled z-positions, reflecting adjustments made to align the scan data based on the polynomial fit.

#### Press ▶︎ to Level  3D Scan:

In [63]:
print("🟧 Running", end='\r')
leveledScan = levelScan(heightMap, mask, xAxisArray, filledScan)
print("✅ Success ")

#### Press ▶︎ to Display Leveled Scan :

In [ ]:
# Create a figure with a specific size
fig, ax = plt.subplots(figsize=(1, 8))  # Width, height in inches

cax = plt.imshow(leveledScan.astype(float), 
                    interpolation ='nearest', origin ='lower',cmap='viridis', aspect = 'auto')  # 'viridis' colormap
#cbar = plt.colorbar()  # Add a color bar for reference
cbar = plt.colorbar(fraction=0.5, pad=0.04) 
# Set the colorbar label
cbar.set_label('Height Value (mm)')

# Add x and y axis labels
plt.xlabel('Y Axis')
plt.ylabel('X Axis')

# Hide the x and y ticks
plt.xticks([])
plt.yticks([])
plt.show()

## Calculate Bead Cross Sectional Area:
This section calculates the cross sectional area of the printed bead along bead length. The calculateBeadArea function computes the area of beads by integrating the leveled heightmap across the x-positions for each line in a 3D scan. It first ensures all input arrays are numpy arrays of type float and initializes an array to store the bead areas. For each line in the scan, the function identifies bead indices where the z_mask is False and extracts corresponding x and leveled z-positions. It removes any NaN values from these positions and uses numerical integration (trapezoidal rule) to calculate the area under the curve of the leveled z-positions. The function returns an array containing the computed bead areas for each line, with areas set to zero if no valid points were found for integration. The trapezoidal rule is a numerical method used to approximate the definite integral of a function. It works by dividing the area under the curve into a series of trapezoids and summing their areas. For a function $ f(x) $ evaluated at $ n $ points $ x_0, x_1, \ldots, x_n $ over an interval $[a, b]$, the integral is approximated as:


$$
\int_a^b f(x) \, dx \approx \frac{b - a}{2n} \left[ f(x_0) + 2 \sum_{i=1}^{n-1} f(x_i) + f(x_n) \right]
$$


This formula can be simplified for equally spaced points where $ \Delta x = \frac{b - a}{n} $:


$$
\int_a^b f(x) \, dx \approx \frac{\Delta x}{2} \left[ f(x_0) + 2 \sum_{i=1}^{n-1} f(x_i) + f(x_n) \right]
$$


By using this rule, the area under the curve is estimated with greater accuracy as the number of points $ n $ increases.



#### Press ▶︎ to Calculate Bead Cross-Sectional Area:

In [65]:
print("🟧 Running", end='\r')
beadCrossSectionArea = calculateBeadArea(heightMap, mask, xAxisArray, leveledScan)
print("✅ Success ")

#### Press ▶︎ to Display Bead Area Across the Bead Length:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=yAxisArray, y=beadCrossSectionArea, mode='lines', showlegend=True))
# Update layout
fig.update_layout(xaxis_title='X Axis (mm)',
                  yaxis_title='Bead Area (mm^2)')

# Show the plot
fig.show()


## Resample Bead Area To Fit Feedback Data:
The sample spacing varies between the Aerotech feedback and the 3D scan, necessitating resampling of the x spacing to be equal to that of the Aerotech system. The resampleBeadArea function resamples the bead cross-sectional area data based on new axis feedback data points. It starts by converting the input arrays to numeric types, handling non-numeric values by replacing them with NaNs. It then removes any NaN values from the beadArea and axisFeedbackData arrays. An interpolator function is created using the axisScan and beadArea data, with linear interpolation and extrapolation enabled for out-of-bounds values. Finally, the function performs interpolation using the new axisFeedbackData points and returns the interpolated bead area values.

#### Press ▶︎ to Resample Bead Area Array:

In [67]:
print("🟧 Running", end='\r')
yAxisArray = [float(x) for x in yAxisArray]
yAxisArray = np.array(yAxisArray)+(-1*yAxisArray[0])
resampledBeadArea = resampleBeadArea(beadCrossSectionArea,yAxisArray,(xAxisPositionCommand[512:]-xAxisPositionCommand[0])/50000)
print("✅ Success ")

## Plot Various Feedback Signals Vs. Time:
#### Press ▶︎ to Display Plot:

In [ ]:
startOffset = 512
# Create a figure with a solid line
fig = go.Figure()

# Add a solid line trace
fig.add_trace(go.Scatter(x=experimentTimeArray[startOffset:]/1000, y=resampledBeadArea, mode='lines', name='Bead Area (mm^2)', showlegend=True))

fig.add_trace(go.Scatter(x=experimentTimeArray[startOffset:]/1000, y=(loadCellVoltageSignal[startOffset:]+(-1*loadCellVoltageSignal[startOffset])), mode='lines', name='Load Cell Signal (V)', showlegend=True))

fig.add_trace(go.Scatter(x=experimentTimeArray[startOffset:]/1000, y=thermocoupleVoltageSignal[startOffset:], mode='lines', name='Temperature Signal (V)', showlegend=True))

fig.add_trace(go.Scatter(x=experimentTimeArray[startOffset:]/1000, y=pressureSensorVoltageSignal[startOffset:], mode='lines', name='Pressure Signal (V)', showlegend=True))

# Update layout
fig.update_layout(title='Feedback Signals Vs. Time',
                  xaxis_title='Time (s)',
                  yaxis_title='Value')

# Show the plot
fig.show()

## [🏠 Home](../../../welcomePage.ipynb)